# Pyspark MNIST example

### Based on https://github.com/abulbasar/pyspark-examples

In [ ]:
import findspark
findspark.init()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pyspark
from pyspark.sql import SQLContext, SparkSession
spark = SparkSession \
        .builder \
        .master('spark://Tux:7077') \
        .appName("sparkFromJupyter") \
        .getOrCreate()
sqlContext = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)
sc=spark.sparkContext
print("Spark Version: " + spark.version)
print("PySpark Version: " + pyspark.__version__)

In [ ]:
spark.sparkContext.uiWebUrl

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [ ]:
df_training = (spark
               .read
               .options(header = False, inferSchema = True)
               .csv("./data/mnist_train.csv"))

In [ ]:
df_training.printSchema()

In [ ]:
df_training.count()

In [ ]:
df_training.take(1)

In [ ]:
print("No of columns: ", len(df_training.columns), df_training.columns)


In [ ]:
feature_culumns = ["_c" + str(i+1) for i in range(784)]
#print(feature_culumns)

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
vectorizer = VectorAssembler(inputCols=feature_culumns, outputCol="features")
training = (vectorizer
            .transform(df_training)
            .select("_c0", "features")
            .toDF("label", "features")
            .cache())
training.show()

In [ ]:
a = training.first().features.toArray()
type(a)

In [ ]:
plt.imshow(a.reshape(28, 28), cmap="Greys")

In [ ]:
images = training.sample(False, 0.01, 1).take(25)
fig, _ = plt.subplots(5, 5, figsize = (10, 10))
for i, ax in enumerate(fig.axes):
    r = images[i]
    label = r.label
    features = r.features
    ax.imshow(features.toArray().reshape(28, 28), cmap = "Greys")
    ax.set_title("True: " + str(label))

plt.tight_layout()
    

In [ ]:
counts = training.groupBy("label").count()

In [ ]:
counts_df = counts.rdd.map(lambda r: {"label": r['label'], 
                                     "count": r['count']}).collect()
pd.DataFrame(counts_df).set_index("label").sort_index().plot.bar()

In [ ]:
df_testing = (spark
              .read
              .options(header = False, inferSchema = True)
              .csv("data/mnist_test.csv"))
testing = (vectorizer
           .transform(df_testing)
           .select("_c0", "features")
           .toDF("label", "features")
           .cache())

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(featuresCol="features", 
                        labelCol="label", 
                        regParam=0.1, 
                        elasticNetParam=0.1, 
                        maxIter=10000)

In [ ]:
lr_model = lr.fit(training)

In [ ]:
from pyspark.sql.functions import *

In [ ]:
test_pred = lr_model.transform(testing).withColumn("matched", expr("label == prediction"))
test_pred.show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", 
                                               predictionCol="prediction", 
                                               metricName="accuracy")

In [ ]:
evaluator.evaluate(test_pred)

In [ ]:
(test_pred
 .withColumn("matched", expr("cast(matched as int)"))
 .groupby("label")
 .agg(avg("matched"))
 .orderBy("label")
 .show())

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [ ]:
layers = [784, 100, 20, 10]
perceptron = MultilayerPerceptronClassifier(maxIter=1000, layers=layers, blockSize=128, seed=1234)
perceptron_model = perceptron.fit(training)

In [ ]:
from time import time

In [ ]:
start_time = time()
perceptron_model = perceptron.fit(training)
test_pred = perceptron_model.transform(testing)
print("Accuracy:", evaluator.evaluate(test_pred))
print("Time taken: %d" % (time() - start_time))